In [26]:
#Libraries
import pandas as pd
import numpy as np
from datetime import dt
import math
import os

# Import code developed and functions
from feature_extraction import preprocess, flatten, label, select_group

# Ignore warnings of appending dataframes
import warnings
warnings.simplefilter('ignore')  

ImportError: cannot import name 'dt' from 'datetime' (c:\Users\mverd\anaconda3\envs\tensorflow-gpu\lib\datetime.py)

In [ ]:
path = 'C:\\Users\\mverd\\Desktop\\IMD\\ESSEX\\PredictingEmpathy\\rawdata\\'
file = 'test\\Participant0059.tsv'

In [50]:
df_table = pd.read_table(path + file,sep='\t',low_memory=False)

# Remove calibration points in recording
startPoints = df_table[df_table['Event']=='ImageStimulusStart'].index.values.astype(int)
endPoints = df_table[df_table['Event']=='ImageStimulusEnd'].index.values.astype(int)

# Store only image stimulus
df = pd.DataFrame()

for i in range(len(startPoints)):
    start = startPoints[i]
    end = endPoints[i]

    trial = df_table.iloc[start:end+1]
    df = pd.concat([df,trial])

# Features we are keeping
df_col = ['Recording timestamp','Participant name',
            'Recording name','Recording duration',
            'Pupil diameter left','Pupil diameter right',
            'Gaze point X (MCSnorm)','Gaze point Y (MCSnorm)',
            'Eye movement type','Gaze event duration',
            'Fixation point X (MCSnorm)','Fixation point Y (MCSnorm)']

# Feature seletion base of data given and purpose of model
df_features = df[df_col]

# Columns that need to be changed from object to float
objColumns = ['Pupil diameter left','Pupil diameter right','Gaze point X (MCSnorm)',
                'Gaze point Y (MCSnorm)','Fixation point X (MCSnorm)','Fixation point Y (MCSnorm)']

# Change (commas) to (decimals) and convert object to float64
for feature in objColumns:
    df_features[feature] = df_features[feature].str.replace(',','.').astype(float)

# Change timestamp to timedelta in miliseconds
df_features['Time'] = pd.to_timedelta(df_features['Recording timestamp'], unit='us')
df_features['Delta Time'] = df_features['Time'].diff() / np.timedelta64(1, 'us')
df_features['Distance'] = np.sqrt(df_features['Gaze point X (MCSnorm)']**2 + df_features['Gaze point Y (MCSnorm)']**2)
df_features['Speed'] = df_features['Distance'] / df_features['Delta Time']

df_features

,Recording timestamp,Participant name,Recording name,Recording duration,Pupil diameter left,Pupil diameter right,Gaze point X (MCSnorm),Gaze point Y (MCSnorm),Eye movement type,Gaze event duration,Fixation point X (MCSnorm),Fixation point Y (MCSnorm),Time,Delta Time,Distance,Speed
1610,12532530,Participant0059,Recording1,88301,NaN,NaN,NaN,NaN,EyesNotFound,75.0,NaN,NaN,0 days 00:00:12.532530,NaN,NaN,NaN
1611,12533294,Participant0059,Recording1,88301,NaN,NaN,NaN,NaN,EyesNotFound,75.0,NaN,NaN,0 days 00:00:12.533294,764.0,NaN,NaN
1612,12541623,Participant0059,Recording1,88301,NaN,NaN,NaN,NaN,EyesNotFound,75.0,NaN,NaN,0 days 00:00:12.541623,8329.0,NaN,NaN
1613,12549985,Participant0059,Recording1,88301,NaN,NaN,NaN,NaN,EyesNotFound,75.0,NaN,NaN,0 days 00:00:12.549985,8362.0,NaN,NaN
1614,12558306,Participant0059,Recording1,88301,NaN,NaN,NaN,NaN,EyesNotFound,75.0,NaN,NaN,0 days 00:00:12.558306,8321.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102967,63277840,Participant0059,Recording8,63470,NaN,NaN,NaN,NaN,Unclassified,17.0,NaN,NaN,0 days 00:01:03.277840,3958.0,NaN,NaN
102968,63277840,Participant0059,Recording8,63470,NaN,NaN,NaN,NaN,Unclassified,17.0,NaN,NaN,0 days 00:01:03.277840,0.0,NaN,NaN
102969,63282267,Participant0059,Recording8,63470,NaN,NaN,NaN,NaN,EyesNotFound,8.0,NaN,NaN,0 days 00:01:03.282267,4427.0,NaN,NaN
102970,63290558,Participant0059,Recording8,63470,NaN,NaN,0.4547,0.4617,Unclassified,8.0,NaN,NaN,0 days 00:01:03.290558,8291.0,0.648012,0.000078


In [51]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95250 entries, 1610 to 102971
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   Recording timestamp         95250 non-null  int64          
 1   Participant name            95250 non-null  object         
 2   Recording name              95250 non-null  object         
 3   Recording duration          95250 non-null  int64          
 4   Pupil diameter left         22585 non-null  float64        
 5   Pupil diameter right        21218 non-null  float64        
 6   Gaze point X (MCSnorm)      58133 non-null  float64        
 7   Gaze point Y (MCSnorm)      58133 non-null  float64        
 8   Eye movement type           95243 non-null  object         
 9   Gaze event duration         95243 non-null  float64        
 10  Fixation point X (MCSnorm)  31628 non-null  float64        
 11  Fixation point Y (MCSnorm)  31628 non